In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

def split_dataset(data, target_column_name, test_size=0.2, val_size=0.1, random_state=None):
    """
    Splits the dataset into features and target variables, then further splits into 
    training, validation, and test sets.
    
    Args:
    data (pd.DataFrame): The dataset to split.
    target_column_name (str): The name of the target column.
    test_size (float): Proportion of the dataset to include in the test split.
    val_size (float): Proportion of the dataset to include in the validation split.
    random_state (int, optional): Controls the shuffling applied to the data before 
                                  applying the split. Pass an int for reproducible output.
    
    Returns:
    X_train (pd.DataFrame): Training features.
    X_val (pd.DataFrame): Validation features.
    X_test (pd.DataFrame): Test features.
    y_train (pd.Series): Training target variable.
    y_val (pd.Series): Validation target variable.
    y_test (pd.Series): Test target variable.
    """
    # Ensure the target column is in the dataframe
    if target_column_name not in data.columns:
        raise ValueError(f"Target column '{target_column_name}' not found in dataset.")
    
    # Split the dataset into features and target
    X = data.drop(columns=[target_column_name])
    y = data[target_column_name]
    
    # First, split into train and test sets
    X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=test_size, random_state=random_state, stratify=y)
    
    # Calculate the proportion of validation set with respect to the temporary set (remaining data after train split)
    val_proportion = val_size / (1 - test_size)
    
    # Split the temporary set into validation and test sets
    X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=test_size, random_state=random_state, stratify=y_temp)
    
    return X_train, X_val, X_test, y_train, y_val, y_test

# Example usage:
# Assuming you have a DataFrame `df` and the target column is 'target'
# df = pd.read_csv('your_dataset.csv')
# X_train, X_val, X_test, y_train, y_val, y_test = split_dataset(df, 'target')


In [59]:
test_result = {}

In [57]:
def save_test_result(model_name, score):
    test_result[model_name] = score

In [2]:
df = pd.read_csv(r'E:\Project Thesis\Data set\final\Notebook\data.csv')
target_column_name = 'quality'

X_train, X_val, X_test, y_train, y_val, y_test = split_dataset(df ,target_column_name )

In [3]:
print(X_train.shape)
print(X_val.shape)
print(X_test.shape) 

(1160, 13)
(232, 13)
(59, 13)


In [4]:
y_train= y_train - 1
y_val= y_val - 1
y_test = y_test - 1

In [10]:
import pandas as pd
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score
import optuna
import joblib

In [15]:
def train_and_tune_model(X_train, y_train, X_val, y_val, classifier_name):
    def objective(trial):
        if classifier_name == "K-Neighbors Classifier":
            n_neighbors = trial.suggest_int('n_neighbors', 1, 50)
            model = KNeighborsClassifier(n_neighbors=n_neighbors)
        elif classifier_name == "Decision Tree":
            max_depth = trial.suggest_int('max_depth', 1, 50)
            model = DecisionTreeClassifier(max_depth=max_depth)
        elif classifier_name == "Random Forest Classifier":
            n_estimators = trial.suggest_int('n_estimators', 50, 500)
            max_depth = trial.suggest_int('max_depth', 1, 50)
            model = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth)
        elif classifier_name == "XGBClassifier":
            ''' n_estimators = trial.suggest_int('n_estimators', 50, 500)
            max_depth = trial.suggest_int('max_depth', 2, 15),
            learning_rate = trial.suggest_loguniform('learning_rate', 0.01, 0.3)
            model = XGBClassifier(n_estimators=n_estimators, max_depth=max_depth, learning_rate=learning_rate,tree_method='gpu_hist', gpu_id=0, use_label_encoder=False)'''
            
            param = {
            'max_depth': trial.suggest_int('max_depth', 2, 15),
            'subsample': trial.suggest_discrete_uniform('subsample', 0.6, 1.0, 0.05),
            'n_estimators': trial.suggest_int('n_estimators', 1000, 10000, 100),
            'eta': trial.suggest_discrete_uniform('eta', 0.01, 0.1, 0.01),
            'reg_alpha': trial.suggest_int('reg_alpha', 1, 50),
            'reg_lambda': trial.suggest_int('reg_lambda', 5, 100),
            'min_child_weight': trial.suggest_int('min_child_weight', 2, 20),
            "colsample_bytree": trial.suggest_float("colsample_bytree", 0.1, 1.0),
             } 
            model = XGBClassifier(random_state=42, 
                             tree_method='hist', 
                             
                             predictor="cpu_predictor"
                             ,**param ) 
            
        elif classifier_name == "CatBoostClassifier":
            n_estimators = trial.suggest_int('n_estimators', 50, 500)
            depth = trial.suggest_int('depth', 1, 10)
            learning_rate = trial.suggest_float('learning_rate', 0.01, 0.3)
            model = CatBoostClassifier(n_estimators=n_estimators, depth=depth, learning_rate=learning_rate, task_type='GPU', verbose=0)
        else:
            raise ValueError("Unknown classifier: {}".format(classifier_name))
        
        model.fit(X_train, y_train)
        y_pred = model.predict(X_val)
        accuracy = accuracy_score(y_val, y_pred)
        return accuracy

    study = optuna.create_study(direction='maximize')
    study.optimize(objective, n_trials=50)

    best_params = study.best_params
    if classifier_name == "K-Neighbors Classifier":
        model = KNeighborsClassifier(**best_params)
    elif classifier_name == "Decision Tree":
        model = DecisionTreeClassifier(**best_params)
    elif classifier_name == "Random Forest Classifier":
        model = RandomForestClassifier(**best_params)
    elif classifier_name == "XGBClassifier":
        model = XGBClassifier(**best_params,tree_method='hist')
    elif classifier_name == "CatBoostClassifier":
        model = CatBoostClassifier(**best_params,task_type='GPU',devices='0', verbose=0)
    
    model.fit(X_train, y_train)
    return model, best_params

In [16]:
classifiers = {
        "K-Neighbors Classifier": KNeighborsClassifier(),
        "Decision Tree": DecisionTreeClassifier(),
        "Random Forest Classifier": RandomForestClassifier(),
        "XGBClassifier": XGBClassifier(),
        "CatBoostClassifier": CatBoostClassifier()
    }

for classifier_name in classifiers.keys():
    print(f"Training {classifier_name}...")
    model, best_params = train_and_tune_model(X_train, y_train, X_val, y_val, classifier_name)
    print(f"Best parameters for {classifier_name}: {best_params}")
    
    # Save the best model
    model_filename = f"best_{classifier_name.replace(' ', '_').lower()}.pkl"
    joblib.dump(model, model_filename)
    print(f"Model saved as {model_filename}")
    
    # Evaluate the model on the test set
    y_pred = model.predict(X_test)
    test_accuracy = accuracy_score(y_test, y_pred)
    print(f"Test accuracy for {classifier_name}: {test_accuracy:.4f}")

[I 2024-07-03 13:03:31,505] A new study created in memory with name: no-name-82ac96a1-ca38-4f9d-b7ba-5d8e1e1ca4a4


Training XGBClassifier...


C:\Users\admin\AppData\Local\Temp\ipykernel_5148\487667124.py:21: FutureWarning: suggest_discrete_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., step=...) instead.
  'subsample': trial.suggest_discrete_uniform('subsample', 0.6, 1.0, 0.05),
C:\Users\admin\AppData\Local\Temp\ipykernel_5148\487667124.py:22: FutureWarning: suggest_int() got {'step'} as positional arguments but they were expected to be given as keyword arguments.
  'n_estimators': trial.suggest_int('n_estimators', 1000, 10000, 100),
C:\Users\admin\AppData\Local\Temp\ipykernel_5148\487667124.py:23: FutureWarning: suggest_discrete_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., step=...) instead.
  'eta': trial.suggest_discrete_uniform('eta', 0.01, 0.1, 0.01),
e:\Project Thesis\Data set\final\venv\lib\sit

Best parameters for XGBClassifier: {'max_depth': 6, 'subsample': 0.95, 'n_estimators': 3000, 'eta': 0.05, 'reg_alpha': 5, 'reg_lambda': 19, 'min_child_weight': 6, 'colsample_bytree': 0.6699718653592507}


[I 2024-07-03 13:23:19,431] A new study created in memory with name: no-name-11c7961d-48f7-4517-862f-3ce68af1ac71


Model saved as best_xgbclassifier.pkl
Test accuracy for XGBClassifier: 0.9661
Training CatBoostClassifier...


[I 2024-07-03 13:23:58,327] Trial 0 finished with value: 0.9353448275862069 and parameters: {'n_estimators': 405, 'depth': 6, 'learning_rate': 0.23287424574001783}. Best is trial 0 with value: 0.9353448275862069.
[I 2024-07-03 13:24:05,016] Trial 1 finished with value: 0.9439655172413793 and parameters: {'n_estimators': 225, 'depth': 5, 'learning_rate': 0.2916172117608551}. Best is trial 1 with value: 0.9439655172413793.
[I 2024-07-03 13:24:15,936] Trial 2 finished with value: 0.9310344827586207 and parameters: {'n_estimators': 434, 'depth': 5, 'learning_rate': 0.2988976432792824}. Best is trial 1 with value: 0.9439655172413793.
[I 2024-07-03 13:24:19,232] Trial 3 finished with value: 0.9267241379310345 and parameters: {'n_estimators': 214, 'depth': 1, 'learning_rate': 0.12054150297464668}. Best is trial 1 with value: 0.9439655172413793.
[I 2024-07-03 13:24:27,743] Trial 4 finished with value: 0.9396551724137931 and parameters: {'n_estimators': 94, 'depth': 10, 'learning_rate': 0.20024

Best parameters for CatBoostClassifier: {'n_estimators': 293, 'depth': 8, 'learning_rate': 0.11161795034583433}
Model saved as best_catboostclassifier.pkl
Test accuracy for CatBoostClassifier: 0.9831


In [60]:
classifiers = {
        "K-Neighbors Classifier": KNeighborsClassifier(),
        "Decision Tree": DecisionTreeClassifier(),
        "Random Forest Classifier": RandomForestClassifier(),
        "XGBClassifier": XGBClassifier(),
        "CatBoostClassifier": CatBoostClassifier()}
# Load the saved models and evaluate them on the test set
for classifier_name in classifiers:
    # Load the model
    model_filename = f"best_{classifier_name.replace(' ', '_').lower()}.pkl"
    model = joblib.load(model_filename)
    
    # Make predictions on the test set
    y_pred = model.predict(X_test)
    
    # Evaluate the model
    test_accuracy = accuracy_score(y_test, y_pred)
    save_test_result(model_name= classifier_name, score = test_accuracy)
    print(f"Test accuracy for {classifier_name}: {test_accuracy:.4f}")

Test accuracy for K-Neighbors Classifier: 0.8983
Test accuracy for Decision Tree: 1.0000
Test accuracy for Random Forest Classifier: 1.0000
Test accuracy for XGBClassifier: 0.9661
Test accuracy for CatBoostClassifier: 0.9831


## Training on Deep Learning models

In [50]:
import torch
from sklearn.preprocessing import StandardScaler
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torch.nn.functional as F



In [29]:
def scale(X_train, X_val, X_test, y_train, y_val, y_test):

    X_train_array= X_train.values
    X_val_array=X_val.values
    X_test_array=X_test.values
    y_train_array= y_train.values
    y_val_array=y_val.values 
    y_test_array= y_test.values

    sc = StandardScaler()

    X_train_array = sc.fit_transform(X_train_array)
    X_val_array= sc.transform(X_val_array)
    X_test_array = sc.transform(X_test_array)

    # Convert numpy arrays to PyTorch tensors
    X_train_tensor = torch.tensor(X_train_array, dtype=torch.float32)
    y_train_tensor = torch.tensor(y_train_array, dtype=torch.long)  # Assuming class labels are integers

    # Convert numpy arrays to PyTorch tensors
    X_val_tensor = torch.tensor(X_val_array, dtype=torch.float32)
    y_val_tensor = torch.tensor(y_val_array, dtype=torch.long)  # Assuming class labels are integers

    # Convert numpy arrays to PyTorch tensors
    X_test_tensor = torch.tensor(X_test_array, dtype=torch.float32)
    y_test_tensor = torch.tensor(y_test_array, dtype=torch.long)  # Assuming class labels are integers


    return X_train_tensor, y_train_tensor, X_val_tensor, y_val_tensor, X_test_tensor, y_test_tensor

In [30]:
X_train_tensor, y_train_tensor, X_val_tensor, y_val_tensor, X_test_tensor, y_test_tensor = scale(X_train, X_val, X_test, y_train, y_val, y_test)

## MLP

In [65]:
# Define the MLP model
class MLP(nn.Module):
    def __init__(self, input_dim, hidden_dim1, hidden_dim2,hidden_dim3, output_dim,drop_out_prob):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim1)
        self.fc2 = nn.Linear(hidden_dim1, hidden_dim2)
        self.fc3 = nn.Linear(hidden_dim2, hidden_dim3)
        self.fc4 = nn.Linear(hidden_dim3, output_dim)
        self.relu = nn.ReLU()
        self.Dropout = nn.Dropout(drop_out_prob)

    def forward(self, x):
        out = self.Dropout(self.relu(self.fc1(x)))
        out = self.Dropout(self.relu(self.fc2(out)))
        out = self.Dropout(self.relu(self.fc3(out)))
        out = self.fc4(out)

        return out

In [39]:
# Initialize the model, loss function, and optimizer
input_dim = X_train.shape[1]
hidden_dim1 = 64  # You can change this to experiment with different architectures
hidden_dim2 = 32 
hidden_dim3 = 16
output_dim = len(np.unique(y_train))  # Number of unique classes
model = MLP(input_dim, hidden_dim1,hidden_dim2,hidden_dim3, output_dim, drop_out_prob=0.2)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)  # You can adjust the learning rate


num_epochs = 500
patience = 20  # Number of epochs to wait if validation loss stops improving
best_val_loss = float('inf')
counter = 0  # Counter for early stopping
best_model = None  # Variable to store the best model state_dict

In [40]:
# Training loop with early stopping and model saving
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    loss.backward()
    optimizer.step()
    
    # Validation loss
    model.eval()
    with torch.no_grad():
        val_outputs = model(X_val_tensor)
        val_loss = criterion(val_outputs, y_val_tensor)
    
    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}, Val Loss: {val_loss.item():.4f}')
    
    # Early stopping logic and model saving
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        counter = 0
        # Save the best model state_dict
        best_model = model.state_dict()
    else:
        counter += 1
        if counter >= patience:
            print(f'Early stopping at epoch {epoch+1} with validation loss: {val_loss.item():.4f}')
            break

# Load the best model state_dict back into the model
if best_model is not None:
    model.load_state_dict(best_model)
    print("Best model loaded.")

# Save the best model to a file
torch.save(model.state_dict(), 'best_MLP_model.pth')
print("Best model saved to 'best_MLP_model.pth'.")


Epoch [10/500], Loss: 1.3740, Val Loss: 1.3627
Epoch [20/500], Loss: 1.3181, Val Loss: 1.3008
Epoch [30/500], Loss: 1.2295, Val Loss: 1.1997
Epoch [40/500], Loss: 1.1041, Val Loss: 1.0491
Epoch [50/500], Loss: 0.9439, Val Loss: 0.8683
Epoch [60/500], Loss: 0.7671, Val Loss: 0.7080
Epoch [70/500], Loss: 0.6727, Val Loss: 0.5992
Epoch [80/500], Loss: 0.5827, Val Loss: 0.5189
Epoch [90/500], Loss: 0.5593, Val Loss: 0.4642
Epoch [100/500], Loss: 0.4802, Val Loss: 0.4309
Epoch [110/500], Loss: 0.4592, Val Loss: 0.4101
Epoch [120/500], Loss: 0.4458, Val Loss: 0.3950
Epoch [130/500], Loss: 0.4320, Val Loss: 0.3840
Epoch [140/500], Loss: 0.4243, Val Loss: 0.3751
Epoch [150/500], Loss: 0.3872, Val Loss: 0.3670
Epoch [160/500], Loss: 0.3956, Val Loss: 0.3596
Epoch [170/500], Loss: 0.3819, Val Loss: 0.3520
Epoch [180/500], Loss: 0.3872, Val Loss: 0.3448
Epoch [190/500], Loss: 0.3738, Val Loss: 0.3379
Epoch [200/500], Loss: 0.3667, Val Loss: 0.3323
Epoch [210/500], Loss: 0.3700, Val Loss: 0.3268
E

In [66]:
model = MLP(input_dim, hidden_dim1,hidden_dim2,hidden_dim3, output_dim, drop_out_prob=0.2)

model.load_state_dict(torch.load('best_MLP_model.pth'))

# Evaluate the model on test set
model.eval()
with torch.no_grad():
    outputs = model(X_test_tensor)
    _, predicted = torch.max(outputs, 1)

# Calculate accuracy on test set
correct = (predicted == y_test_tensor).sum().item()
accuracy = correct / len(y_test_tensor) * 100
save_test_result(model_name= 'MLP', score= accuracy)
print(f'Test Accuracy: {accuracy:.2f}%')

Test Accuracy: 94.92%


## TAB PFN

In [44]:
from tabpfn.scripts.transformer_prediction_interface import TabPFNClassifier

In [45]:
classifier = TabPFNClassifier(device='cpu',N_ensemble_configurations=64)
classifier.fit(X_train, y_train, overwrite_warning=True)
y_eval, p_eval = classifier.predict(X_test, return_winning_probability=True)

print('Accuracy', accuracy_score(y_test, y_eval))

We have to download the TabPFN, as there is no checkpoint at  E:\Project Thesis\Data set\final\venv\Lib\site-packages\tabpfn\models_diff/prior_diff_real_checkpoint_n_0_epoch_100.cpkt
It has about 100MB, so this might take a moment.


e:\Project Thesis\Data set\final\venv\lib\site-packages\torch\utils\checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
e:\Project Thesis\Data set\final\venv\lib\site-packages\torch\utils\checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Accuracy 0.9491525423728814


In [67]:
y_eval, p_eval = classifier.predict(X_val, return_winning_probability=True)

save_test_result(model_name='Tab pfn', score =accuracy_score(y_val, y_eval) )

print('Accuracy', accuracy_score(y_val, y_eval))

e:\Project Thesis\Data set\final\venv\lib\site-packages\torch\utils\checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
e:\Project Thesis\Data set\final\venv\lib\site-packages\torch\utils\checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Accuracy 0.9224137931034483


## 1 D CNN for Tabular data

In [47]:
# Model from https://github.com/baosenguo/Kaggle-MoA-2nd-Place-Solution/blob/main/training/1d-cnn-train.ipynb

class OneD_Model(nn.Module):
        def __init__(self, num_features, num_targets, hidden_size):
            super(OneD_Model, self).__init__()
            cha_1 = 256
            cha_2 = 512
            cha_3 = 512

            cha_1_reshape = int(hidden_size/cha_1)
            cha_po_1 = int(hidden_size/cha_1/2)
            cha_po_2 = int(hidden_size/cha_1/2/2) * cha_3

            self.cha_1 = cha_1
            self.cha_2 = cha_2
            self.cha_3 = cha_3
            self.cha_1_reshape = cha_1_reshape
            self.cha_po_1 = cha_po_1
            self.cha_po_2 = cha_po_2

            self.batch_norm1 = nn.BatchNorm1d(num_features)
            self.dropout1 = nn.Dropout(0.1)
            self.dense1 = nn.utils.weight_norm(nn.Linear(num_features, hidden_size))

            self.batch_norm_c1 = nn.BatchNorm1d(cha_1)
            self.dropout_c1 = nn.Dropout(0.1)
            self.conv1 = nn.utils.weight_norm(nn.Conv1d(cha_1,cha_2, kernel_size = 5, stride = 1, padding=2,  bias=False),dim=None)

            self.ave_po_c1 = nn.AdaptiveAvgPool1d(output_size = cha_po_1)

            self.batch_norm_c2 = nn.BatchNorm1d(cha_2)
            self.dropout_c2 = nn.Dropout(0.1)
            self.conv2 = nn.utils.weight_norm(nn.Conv1d(cha_2,cha_2, kernel_size = 3, stride = 1, padding=1, bias=True),dim=None)

            self.batch_norm_c2_1 = nn.BatchNorm1d(cha_2)
            self.dropout_c2_1 = nn.Dropout(0.3)
            self.conv2_1 = nn.utils.weight_norm(nn.Conv1d(cha_2,cha_2, kernel_size = 3, stride = 1, padding=1, bias=True),dim=None)

            self.batch_norm_c2_2 = nn.BatchNorm1d(cha_2)
            self.dropout_c2_2 = nn.Dropout(0.2)
            self.conv2_2 = nn.utils.weight_norm(nn.Conv1d(cha_2,cha_3, kernel_size = 5, stride = 1, padding=2, bias=True),dim=None)

            self.max_po_c2 = nn.MaxPool1d(kernel_size=4, stride=2, padding=1)

            self.flt = nn.Flatten()

            self.batch_norm3 = nn.BatchNorm1d(cha_po_2)
            self.dropout3 = nn.Dropout(0.2)
            self.dense3 = nn.utils.weight_norm(nn.Linear(cha_po_2, num_targets))

        def forward(self, x):

            x = self.batch_norm1(x)
            x = self.dropout1(x)
            x = F.celu(self.dense1(x), alpha=0.06)

            x = x.reshape(x.shape[0],self.cha_1,
                          self.cha_1_reshape)

            x = self.batch_norm_c1(x)
            x = self.dropout_c1(x)
            x = F.relu(self.conv1(x))

            x = self.ave_po_c1(x)

            x = self.batch_norm_c2(x)
            x = self.dropout_c2(x)
            x = F.relu(self.conv2(x))
            x_s = x

            x = self.batch_norm_c2_1(x)
            x = self.dropout_c2_1(x)
            x = F.relu(self.conv2_1(x))

            x = self.batch_norm_c2_2(x)
            x = self.dropout_c2_2(x)
            x = F.relu(self.conv2_2(x))
            x =  x * x_s

            x = self.max_po_c2(x)

            x = self.flt(x)

            x = self.batch_norm3(x)
            x = self.dropout3(x)
            x = self.dense3(x)

            return x

In [48]:
num_features = X_train.shape[1]
num_targets = len(np.unique(y_train))
hidden_size = 4096
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model2 = OneD_Model(num_features, num_targets, hidden_size).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model2.parameters(), lr=0.001)  # You can adjust the learning rate

num_epochs = 500
patience = 10  # Number of epochs to wait if validation loss stops improving
best_val_loss = float('inf')
counter = 0  # Counter for early stopping
best_model = None  # Variable to store the best model state_dict

e:\Project Thesis\Data set\final\venv\lib\site-packages\torch\nn\utils\weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.


In [51]:
# Training loop with early stopping
for epoch in range(num_epochs):
    model2.train()
    optimizer.zero_grad()
    outputs = model2(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    loss.backward()
    optimizer.step()
    
    # Validation loss
    model2.eval()
    with torch.no_grad():
        val_outputs = model2(X_val_tensor)
        val_loss = criterion(val_outputs, y_val_tensor)
    
    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}, Val Loss: {val_loss.item():.4f}')
    
    # Early stopping logic
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        counter = 0
        # Save the best model state_dict
        best_model = model2.state_dict()
    else:
        counter += 1
        if counter >= patience:
            print(f'Early stopping at epoch {epoch+1} with validation loss: {val_loss.item():.4f}')
            break

# Load the best model state_dict back into the model
if best_model is not None:
    model2.load_state_dict(best_model)
    print("Best model loaded.")

# Save the best model to a file
torch.save(model2.state_dict(), 'best_1D_model.pth')
print("Best model saved to 'best_1D_model.pth'.")

Epoch [10/500], Loss: 0.2326, Val Loss: 1.0660
Epoch [20/500], Loss: 0.1875, Val Loss: 0.3627
Epoch [30/500], Loss: 0.1723, Val Loss: 0.2642
Epoch [40/500], Loss: 0.1578, Val Loss: 0.2926
Early stopping at epoch 41 with validation loss: 0.3150
Best model loaded.
Best model saved to 'best_1D_model.pth'.


In [68]:
model2.load_state_dict(torch.load('best_1D_model.pth'))

# Evaluate the model on test set
model2.eval()
with torch.no_grad():
    outputs = model2(X_test_tensor)
    _, predicted = torch.max(outputs, 1)

# Calculate accuracy on test set
correct = (predicted == y_test_tensor).sum().item()
accuracy = correct / len(y_test_tensor) * 100
save_test_result(model_name='1D CNN', score=accuracy)
print(f'Test Accuracy: {accuracy:.2f}%')

Test Accuracy: 94.92%


In [72]:
result = pd.DataFrame.from_dict(test_result, orient='index', columns=['score' ])

In [73]:
result.head(10)

,score
K-Neighbors Classifier,0.898305
Decision Tree,1.000000
Random Forest Classifier,1.000000
XGBClassifier,0.966102
CatBoostClassifier,0.983051
MLP,94.915254
Tab pfn,0.922414
1D CNN,94.915254
